# Notebook for Train Cyclical Stochastic Gradient MCMC with Synthetic Dataset 

Centro de Investigación en Matemáticas, A.C.  (CIMAT)  
mario.canche@cimat.mx  
23/April/2021

## First Conect with Personal Drive

In [ ]:
# Montamos el Drive al Notebook
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Verificamos el directorio en el que nos encontramos
!pwd
!ls

In [ ]:
# Cambiamos de directorio al Drive
import os
os.chdir("drive/My Drive/PruebasCOLAB2/OF-PathPred/tests")
!ls

## Later, Train Model Cyclical SG-MCMC

In [ ]:
# Imports
import sys,os
''' TF_CPP_MIN_LOG_LEVEL
0 = all messages are logged (default behavior)
1 = INFO messages are not printed
2 = INFO and WARNING messages are not printeds
3 = INFO, WARNING, and ERROR messages are not printed
'''
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
sys.path.append('../')
import math,numpy as np
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
print('[INF] Tensorflow version: ',tf.__version__)
tf.test.gpu_device_name()
# Important imports
import path_prediction.batches_data
import matplotlib.pyplot as plt
from path_prediction.model import TrajectoryEncoderDecoder, Model_Parameters
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from path_prediction.datasets_utils import setup_loo_experiment_synthec, get_testing_batch_synthec
from path_prediction.plot_utils import plot_training_data,plot_training_results
from path_prediction.testing_utils import evaluation_minadefde,evaluation_qualitative,plot_comparisons_minadefde
from path_prediction.training_utils import BDL_training_loop
from path_prediction.training_utils import Experiment_Parameters

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices)>0:
    print('[INF] Using GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("[INF] Using CPU")

In [ ]:
# Load the default parameters
experiment_parameters = Experiment_Parameters(add_social=False,add_kp=False,obstacles=False)

#dataset_dir   = "../datasets/"
#dataset_names = ['eth-hotel','eth-univ','ucy-zara01','ucy-zara02','ucy-univ']

dataset_dir   = "../datasets_synthetic/"
dataset_names = ['testing_toy.txt','training_toy.txt']

In [ ]:
!ls

In [ ]:
# Seed for Random Numpy
seed = 0
# Load the dataset and perform the split
idTest = 0
training_data,validation_data,test_data = setup_loo_experiment_synthec('.',dataset_dir,dataset_names,idTest,experiment_parameters,use_pickled_data=False,seed=seed)

In [ ]:
#############################################################
# Model parameters
model_parameters = Model_Parameters(add_attention=True,add_kp=experiment_parameters.add_kp,add_social=experiment_parameters.add_social,output_representation=experiment_parameters.output_representation)
model_parameters.num_epochs     = 35
model_parameters.output_var_dirs= 0
model_parameters.is_mc_dropout  = False # poner a False
model_parameters.initial_lr     = 0.085
model_parameters.batch_size     = 256

model_parameters.beta = 0.80 # Thresh
model_parameters.datasize = 10800 # datasize of training
model_parameters.M = 4 # number of cycles
model_parameters.alpha = 1
model_parameters.temperature = 1./model_parameters.datasize


In [ ]:
# Get the necessary data
train_data = tf.data.Dataset.from_tensor_slices(training_data)
val_data   = tf.data.Dataset.from_tensor_slices(validation_data)
test_data  = tf.data.Dataset.from_tensor_slices(test_data)


In [ ]:
# Form batches
batched_train_data = train_data.batch(model_parameters.batch_size)
batched_val_data   = val_data.batch(model_parameters.batch_size)
batched_test_data  = test_data.batch(model_parameters.batch_size)


In [ ]:
# Seed for Random Tensorflow
tf.random.set_seed(0)

# Model
tj_enc_dec = TrajectoryEncoderDecoder(model_parameters)

In [ ]:
# Checkpoints
checkpoint_dir   = './training_checkpoints'
checkpoint_prefix= os.path.join(checkpoint_dir, "ckpt")
checkpoint       = tf.train.Checkpoint(optimizer=tj_enc_dec.optimizer,
                                        encoder=tj_enc_dec.enc,
                                        decoder=tj_enc_dec.dec,
                                        enctodec=tj_enc_dec.enctodec,
                                        obs_classif=tj_enc_dec.obs_classif)

In [ ]:
# Training
perform_training = True
plot_training    = True
if perform_training==True:
    print("[INF] Training the model")
    train_loss_results,val_loss_results,val_metrics_results,__ = BDL_training_loop(tj_enc_dec,batched_train_data,batched_val_data,model_parameters,checkpoint,checkpoint_prefix)
    if plot_training==True:
        plot_training_results(train_loss_results,val_loss_results,val_metrics_results)


In [ ]:
# Testing
# Restoring the latest checkpoint in checkpoint_dir
print("[INF] Restoring last model")
status = checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
from testing_utils import predict_from_batch

# Qualitative testing
qualitative = True
if qualitative==True:
    print("[INF] Qualitative testing")
    for i in range(10):
        batch = get_testing_batch_synthec(test_data,dataset_dir+dataset_names[idTest])
        traj_obs,traj_gt,traj_pred,neighbors,__ = predict_from_batch(tj_enc_dec,batch,model_parameters)

        # Plots the trajectories
        plt.figure(figsize=(10,10))
        for j in range(len(traj_obs)):

            obs = traj_obs[j] # Trajectories Obs
            gt = np.vstack([traj_obs[j][-1], traj_gt[j]]) # Ground Thrue
            pred = np.vstack([traj_obs[j][-1], traj_pred[j].numpy()[0]]) # Trajectories Pred

            plt.plot(obs[:,0], obs[:,1],'r',label= 'Observations')
            plt.plot(gt[:,0], gt[:,1],'b',label= 'Ground truth')
            plt.plot(pred[:,0], pred[:,1],'g',label= 'Prediction')

            # Axes
            plt.xlim(-1.3,1.3)
            plt.ylim(-1.15,1.15)
        plt.legend(['Observations', 'Ground truth', 'Prediction'])
        plt.show()

## Finally, Load Weights from Cyclical SG-MCMC

In [ ]:
import pickle
pickle_in = open('training_cSGMCMC_weights.pickle',"rb")
weights = pickle.load(pickle_in)

In [ ]:
len(weights)

In [ ]:
for www in weights:
    print(www[0], www[1][2][6])

In [ ]:
from matplotlib.lines import Line2D

colores = ["green",
           "yellow",
           "pink",
           "purple",
           "cyan",
           "orange",
           "olive",
           "brown",
           "grey",
           "black",
           "magenta",
           "white",
           "gold",
           "lightcoral",
           "darkred",
           "hotpink",
           "chocolate",
           "lightgray",
           "palegreen"
          ]

custom_lines = [Line2D([0],[0],color="purple",marker="s", markersize=7, linestyle=""),
                    Line2D([0],[0],color="red",lw=2),
                    Line2D([0],[0],color="blue",lw=2),
                    Line2D([0],[0],color="green",lw=2),
                    
                    Line2D([0],[0],color="yellow",lw=2),
                    Line2D([0],[0],color="pink",lw=2),
                    Line2D([0],[0],color="purple",lw=2),
                    Line2D([0],[0],color="cyan",lw=2),
                    Line2D([0],[0],color="orange",lw=2),
                    Line2D([0],[0],color="olive",lw=2),
                    Line2D([0],[0],color="brown",lw=2)
                   ]

In [ ]:
from plot_utils import plot_gt_preds, plot_background, plot_neighbors, plot_attention
from batches_data import get_batch
from traj_utils import relative_to_abs

#from testing_utils import compute_kde_nll2

# Qualitative testing
num_print_test = 30
model_parameters.output_var_dirs= 0
qualitative = True
if qualitative==True:
    print("[INF] Qualitative testing")
    for it in range(num_print_test):
        batch = get_testing_batch_synthec(test_data,dataset_dir+dataset_names[idTest])
        print("batch: ", batch['obs_traj'].shape)
        
        # Plot ground truth and predictions
        plt.subplots(1,1,figsize=(10,10))
        ax = plt.subplot(1,1,1)

        #-----------------------------------------------------------------------
        batch_inputs, batch_targets = get_batch(batch, model_parameters)

        # Iterate the diference weights for the model
        cont = 0
        for w_ind, weight in weights:
          # Perform prediction
          pred_traj, _ = tj_enc_dec.predict_cSGMCMC(batch_inputs,batch_targets.shape[1],weight)

          traj_obs      = []
          traj_gt       = []
          traj_pred     = []
          neighbors     = []
          # Cycle over the trajectories of the bach
          for i, (obs_traj_gt, pred_traj_gt, neighbors_gt) in enumerate(zip(batch["obs_traj"], batch["pred_traj"], batch["obs_neighbors"])):
              this_pred_out_abs_set = []

              if (pred_traj[i,0].shape[0] == model_parameters.pred_len):
                  this_pred_out     = pred_traj[i,0,:, :2]
                  # Convert it to absolute (starting from the last observed position)
                  this_pred_out_abs = relative_to_abs(this_pred_out, obs_traj_gt[-1])
                  this_pred_out_abs_set.append(this_pred_out_abs)
               #   print("this_pred_out_abs: ", this_pred_out_abs.shape)
              this_pred_out_abs_set = tf.stack(this_pred_out_abs_set,axis=0)
              # TODO: tensors instead of lists?
              # Keep all the trajectories
              traj_obs.append(obs_traj_gt)
              traj_gt.append(pred_traj_gt)
              traj_pred.append(this_pred_out_abs_set)
              neighbors.append(neighbors_gt)

          #-----------------------------------------------------------------------
          # Plot ground truth and predictions
          if cont == 0:
            pred_out = np.array(traj_pred)
          else:
            pred_out = np.concatenate([pred_out, np.array(traj_pred)],axis=1)
          cont += 1
          obs = traj_obs[j] # Trajectories Obs
          gt = np.vstack([traj_obs[j][-1], traj_gt[j]]) # Ground Thrue
          pred = np.vstack([traj_obs[j][-1], traj_pred[j].numpy()[0]]) # Trajectories Pred

          plt.plot(obs[:,0], obs[:,1],'r')
          plt.plot(gt[:,0], gt[:,1],'b')
          plt.plot(pred[:,0], pred[:,1],"--", label='Prediction C'+str(w_ind+1) , color=colores[int(w_ind)])

          # Axes
          plt.xlim(-1.3,1.3)
          plt.ylim(-1.15,1.15)

        legends = ['Prediction C'+str(ciclo+1) for ciclo in range(int(w_ind)+1)]
        plt.legend(custom_lines,['Observations', 'Ground truth']+legends)
        plt.savefig('images/temp_'+str(it)+'.pdf')
        plt.show()

#        kde_nll, timestep_kde_nll = compute_kde_nll2(pred_out, np.array(traj_gt))
#        print("kde_nll: ", kde_nll)
#        print("timestep_kde_nll: ", timestep_kde_nll)
